In [50]:
%matplotlib inline
import gc
import math
import pandas as pd
import numpy as np
import time, multiprocessing
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from tqdm import tqdm
import seaborn as sns

from rerf.RerF import fastPredict, fastPredictPost, fastRerF
from rerf.rerfClassifier import rerfClassifier

from scipy.stats import bernoulli
from scipy import stats
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import balanced_accuracy_score
from sklearn import preprocessing

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier


gc.enable()

import warnings
warnings.simplefilter('ignore')

In [51]:
dsets = ['ECG200','UWaveGestureLibraryX','Phoneme']
dset = dsets[2]

In [52]:
test_data = np.genfromtxt(f'/mnt/c/Users/Ronan Perry/Downloads/{dset}/{dset}_TEST.txt')
train_data = np.genfromtxt(f'/mnt/c/Users/Ronan Perry/Downloads/{dset}/{dset}_TRAIN.txt')

In [53]:
le = preprocessing.LabelEncoder()

y_test = test_data[:,0]
y_test = le.fit_transform(y_test)
X_test = test_data[:,1:]

y_train = train_data[:,0]
y_train = le.fit_transform(y_train)
X_train = train_data[:,1:]

In [54]:
print(X_test.shape, X_train.shape)

(1896, 1024) (214, 1024)


In [56]:
print(f'{X_train.shape}')
n_features = X_train.shape[1]

(214, 1024)


In [74]:
## Setup for run, define classifiers to train and test
names = {"Log. Reg": "#a6cee3", 
         "Lin. SVM":"#1f78b4", 
         "SVM":"#b2df8a", 
         "kNN": "#33a02c", 
         "RF":"#fb9a99", 
         "MLP":"#fdbf6f", 
         "SporF":"#ff7f00", 
         "MF":"#e31a1c"}
         #"CNN":"#cab2d6"}

ncores=1
num_runs=1
n_est=500
mx = 20
mn = 1

classifiers = [
    LogisticRegression(random_state=0, n_jobs=ncores, solver='liblinear'),
    LinearSVC(),
    SVC(C=1.0, kernel='rbf', gamma='auto',random_state=0),
    KNeighborsClassifier(3, n_jobs=ncores),
    RandomForestClassifier(n_estimators=n_est, max_features='auto', n_jobs=ncores),
    MLPClassifier(hidden_layer_sizes=(100, ), random_state=0, max_iter=500),
    rerfClassifier(n_estimators = n_est, projection_matrix = "RerF",
        max_features = 28, n_jobs = ncores),
    rerfClassifier(
        projection_matrix="S-RerF", 
        max_features = "auto",
        n_estimators=n_est,
        n_jobs=ncores,
        image_height=1, 
        image_width=n_features, 
        patch_height_max=1,
        patch_width_max=mx,
        patch_height_min=1,
        patch_width_min=mn,
        )
    ]

In [75]:
mf = classifiers[-1]

In [69]:
#'max_features':[10,20,30], 
parameters = {'max_features':[7,10,15], 'patch_width_min': list(range(1,3)), 'patch_width_max':list(range(5,21,5))}
skf = StratifiedShuffleSplit(n_splits=1)
clf = GridSearchCV(mf, parameters, cv=skf, verbose=1)
clf.fit(X_train, y_train)

Fitting 1 folds for each of 24 candidates, totalling 24 fits


ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [47]:
mf = clf.best_estimator_

In [76]:
mf.fit(X_train, y_train)

rerfClassifier(feature_combinations=1.5, image_height=1, image_width=1024,
               max_depth=None, max_features='auto', min_parent=1,
               n_estimators=500, n_jobs=1, oob_score=False, patch_height_max=1,
               patch_height_min=1, patch_width_max=20, patch_width_min=1,
               projection_matrix='S-RerF', random_state=None)

In [77]:
yhat_test = mf.predict(X_test)
print(f'Accuracy {np.mean(y_test == yhat_test)}')

Accuracy 0.17035864978902954
